<a href="https://colab.research.google.com/github/ProAek11/StrategyTradingFrendersis-AI-System/blob/main/Algorithm_Logical_Criteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm & Logical Criteria of Strategy Trading to Decision Trade Entry.


## ***Pip Install Requestments All Total***

*Pip Install Pandas All Total*

In [ ]:
!pip install pandas
!pip install pandas_datareader

*Pip Install Yahoo Finncial All Total*

In [ ]:
!pip install Yahoo-ticker-downloader
!pip install yfinance
!pip install yahoo-finance
!pip install yahoo-finance fix
!pip install yfinance fix

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 972.6 kB/s eta 0:00:00
  Created wheel for Yahoo-ticker-downloader: filename=Yahoo_ticker_downloader-3.0.1-py3-none-any.whl size=10165 sha256=28ec0ac12f9f167fdc147b4f948ce32e645c6052f87bde787e786115cea5b91b
  Stored in directory: /root/.cache/pip/wheels/d0/e8/c5/096d50b106a31b8631622e87cecf82053bf0887e289624f05c
Successfully built Yahoo-ticker-downloader
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.5 MB/s eta 0:00:00
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7218 sha256=58945d70f6a285294dc997199f6c6968a76ff4f8cd935a7f912e0a55acab98d8
  Stored in directory: /root/.cache/pip/wheels/f6/a9/34/f1aaa343d0861148f79a9df08f380e4dbbdbe27b7ba1e0e84c
Successfully built yahoo-finance


In [ ]:

#CBOE Interest Rate 10 Year T No (^TNX)
#Revised names symbol
usdx =

us10y = ^TNX
gold = GC=F



SyntaxError: ignored

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("your-credentials-file.json", scope)
client = gspread.authorize(creds)

# Open the Google Sheet by title
sheet = client.open("Your Google Sheet Title").sheet1

# Read data from the Google Sheet
data = sheet.get_all_records()


In [ ]:

import yfinance
import pandas_datareader
yfinance.ticker
symbol =
list_symbol =
high_yield_searching = ['^TNX', , , , , , ]
list_sector_symbol = all_sector_symbol
df = yfinance.download.

In [ ]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_01.07.2020-15.07.2023.csv")
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

In [ ]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

df['pivot'] = df.apply(lambda x: pivotid(df, x.name,3,3), axis=1)

## Pivot Candles Vizualisation

In [ ]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[500:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
from scipy.stats import linregress

def detect_flag(candle, backcandles, window, plot_flag=False):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]
    highs = localdf[localdf['pivot'] == 2].high.tail(3).values
    idxhighs = localdf[localdf['pivot'] == 2].high.tail(3).index
    lows = localdf[localdf['pivot'] == 1].low.tail(3).values
    idxlows = localdf[localdf['pivot'] == 1].low.tail(3).index

    if len(highs) == 3 and len(lows) == 3:
        order_condition = (
            (idxlows[0] < idxhighs[0]
            < idxlows[1] < idxhighs[1]
            < idxlows[2] < idxhighs[2])
            or
            (idxhighs[0] < idxlows[0]
             < idxhighs[1] < idxlows[1]
             < idxhighs[2] < idxlows[2]) )

        slmin, intercmin, rmin, _, _ = linregress(idxlows, lows)
        slmax, intercmax, rmax, _, _ = linregress(idxhighs, highs)

        if (order_condition
            and (rmax*rmax)>=0.9
            and (rmin*rmin)>=0.9
            and slmin>=0.0001
            and slmax<=-0.0001):
            #and ((abs(slmin)-abs(slmax))/abs(slmax)) < 0.05):

            if plot_flag:
                fig = go.Figure(data=[go.Candlestick(x=localdf.index,
                open=localdf['open'],
                high=localdf['high'],
                low=localdf['low'],
                close=localdf['close'])])

                fig.add_scatter(x=localdf.index, y=localdf['pointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot")
                fig.add_trace(go.Scatter(x=idxlows, y=slmin*idxlows + intercmin, mode='lines', name='min slope'))
                fig.add_trace(go.Scatter(x=idxhighs, y=slmax*idxhighs + intercmax, mode='lines', name='max slope'))
                fig.update_layout(
                xaxis_rangeslider_visible=False,
                plot_bgcolor='white', # change the background to white
                xaxis=dict(showgrid=True, gridcolor='white'), # change the x-axis grid to white
                yaxis=dict(showgrid=True, gridcolor='white') # change the y-axis grid to white
                )
                fig.show()

            return 1

    return 0

In [ ]:
detect_flag(2189, 30, 3, plot_flag=True)

In [ ]:
df['flag'] = df.index.map(lambda x: detect_flag(x, 35, 3))

In [ ]:
df[df['flag']!=0]